In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile,assemble
from qiskit import Aer, execute
from qiskit.extensions import Initialize
from qiskit.tools.visualization import plot_histogram, plot_bloch_multivector, array_to_latex
from qiskit.quantum_info import partial_trace, Statevector, random_statevector, Operator
from qiskit_textbook.tools import simon_oracle
import numpy as np
import math

In [ ]:
def create_bell_state(circuit, q1, q2):
    circuit.h(q1)
    circuit.cnot(q1, q2)

def create_bell_state(circuit, register):
    circuit.h(register[0])
    circuit.cnot(register[0], register[1])

def teleport_gates(circuit, alice_qbit, alice_ent):
    circuit.cnot(alice_qbit, alice_ent)
    circuit.h(alice_qbit)


def swap(circuit, q1, q2):
    circuit.cnot(q1, q2)
    circuit.cnot(q2,q1)
    circuit.cnot(q1, q2)

# BB84 Protocol

## Trasmissione semplice

Trasmissione perfetta senza intrusione

In [ ]:


qbit = QuantumRegister(1, 'q')
cbit = ClassicalRegister(1, 'c')
qc = QuantumCircuit(qbit, cbit)


qc.h(qbit)
qc.barrier()
qc.h(qbit)

qc.measure(qbit, cbit)

display(qc.draw(output="mpl"))

backend = Aer.get_backend("qasm_simulator")
job = execute(qc, backend,shots=1000)
result = job.result()
plot_histogram(result.get_counts())


Trasmissione dati con intrusione


In [ ]:
qbit = QuantumRegister(1, 'q')
cbit = ClassicalRegister(1, 'c')
qc = QuantumCircuit(qbit, cbit)


qc.h(qbit)
qc.measure(qbit, cbit)
qc.barrier()


qc.h(qbit)
qc.measure(qbit, cbit)



display(qc.draw(output="mpl"))

backend = Aer.get_backend("qasm_simulator")
job = execute(qc, backend,shots=1000)
result = job.result()
plot_histogram(result.get_counts())

## Trasmissione della chiave

In [ ]:
def code_base(bases):
    return np.array([('+' if a == 0 else 'x') for a in bases])


def encoder(bits, bases):
    message = []
    for bit, base in zip(bits, bases):
        qc = QuantumCircuit(1,1)
        if base == 0 and bit == 0:
            pass
        elif base == 0 and bit == 1:
            qc.x(0)
        elif base == 1 and bit == 0:
            qc.h(0)
        elif base == 1 and bit == 1:
            qc.x(0)
            qc.h(0)
        message.append(qc)
    return message


def decoder(message, bases):
    backend = Aer.get_backend("aer_simulator")
    
    decoded_message = []
    for bit, base in zip(message, bases):
        if base == 0:
            bit.measure(0,0)
        elif base == 1:
            bit.h(0)
            bit.measure(0,0)

        qobj = assemble(bit, shots=1, memory=True)
        decoded_message.append(int(backend.run(qobj).result().get_memory()[0]))
    return np.array(decoded_message)


def remove_garbage(a_bases, b_bases, bits):
    return np.array([i for a,b,i  in zip(a_bases, b_bases, bits) if a==b ])

def sample(bits, selection):
    return np.array([bits[np.mod(i, len(bits))] for i in selection])


- 1) Il messaggio viene trasmesso
- 2) Il ricevente decodifica il messaggio
- 2) Su un canale pubblico vengono condivise le basi, vengono rimossi i bit con basi diverse
- 3) Eseguo un sample dei bit per creare la chiave pubblica

In [ ]:
np.random.seed(seed=0)

BIT_SIZE = 30


alice_bits = np.random.randint(2, size=BIT_SIZE)
alice_bases = np.random.randint(2, size=BIT_SIZE)
bob_bases = np.random.randint(2, size=BIT_SIZE)

display(f"Alice sent message:   {alice_bits}")
display(f"Alice bases:          {alice_bases}")
display(f"Bob   bases:          {bob_bases}")

encoded_message = encoder(alice_bits, alice_bases)
decoded_message = decoder(encoded_message, bob_bases)

display(f"Bob received message: {decoded_message}")

bob_key = remove_garbage(alice_bases, bob_bases, decoded_message)
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)

display(f"Alice key:            {alice_key}")
display(f"Bob key:              {bob_key}")
display(f"KEY EQUAL:            {str((alice_key == bob_key).all()).upper()}")


SAMPLE_SIZE = 30
SELECTION = np.random.randint(BIT_SIZE, size=SAMPLE_SIZE)


alice_sample = sample(alice_key, SELECTION)
bob_sample = sample(bob_key, SELECTION)

display(f"Alice sample:         {alice_sample}")
display(f"Bob sample:           {bob_sample}")


In [ ]:
np.random.seed(seed=0)

BIT_SIZE = 30


alice_bits = np.random.randint(2, size=BIT_SIZE)
alice_bases = np.random.randint(2, size=BIT_SIZE)
bob_bases = np.random.randint(2, size=BIT_SIZE)


display(f"Alice sent message:   {alice_bits}")
display(f"Alice bases:          {alice_bases}")
display(f"Bob   bases:          {bob_bases}")


encoded_message = encoder(alice_bits, alice_bases)

#### INIZIO INTRUSIONE  ####


intruder_bases = np.random.randint(2, size=BIT_SIZE)
intercept = decoder(encoded_message, intruder_bases)
display(f"Intruder   bases:     {intruder_bases}")
display(f"Intruder message:     {intercept}")


#### FINE INTRUSIONE ####

decoded_message = decoder(encoder(intercept, intruder_bases), bob_bases)

display(f"Bob received message: {decoded_message}")

bob_key = remove_garbage(alice_bases, bob_bases, decoded_message)
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)

display(f"Alice key:            {alice_key}")
display(f"Bob key:              {bob_key}")
display(f"KEY EQUAL:            {str((alice_key == bob_key).all()).upper()}")


SAMPLE_SIZE = 30
SELECTION = np.random.randint(BIT_SIZE, size=SAMPLE_SIZE)


alice_sample = sample(alice_key, SELECTION)
bob_sample = sample(bob_key, SELECTION)

display(f"Alice sample:         {alice_sample}")
display(f"Bob sample:           {bob_sample}")